Task 1: Fetch Basic Stock Data 10pts
- Prompt the user to input two or more stock symbols.
- Use the API to fetch data for the given symbols.
- Extract and display only the Stock Ticker, Company Name, and Current Market Price

In [5]:
import requests
import json

yahoo_apikey = "TVd8uCH8ud1nnXTKFJgW95WPMAZw48Nm6x73P1Y6"
headers = {"x-api-key" : yahoo_apikey}

url = "https://yfapi.net/v6/finance/quote"

print("Enter 2 or more stock options, separated by commas without spaces")
symbols = input()

querystring = {"symbols" : symbols}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()["quoteResponse"]["result"]
print("---------------------------------------------")
print("TICKER  |     NAME    |  CURRENT MARKET PRICE")
print("---------------------------------------------")
for stock_result in response:
    symbol, name, market_price = stock_result["symbol"], stock_result["longName"], stock_result["regularMarketPrice"]
    print(symbol, "  |  ", name, "  |  ", market_price)


Enter 2 or more stock options, separated by commas without spaces
AAPL,GOOGL
---------------------------------------------
TICKER  |     NAME    |  CURRENT MARKET PRICE
---------------------------------------------
AAPL   |   Apple Inc.   |   240.36
GOOGL   |   Alphabet Inc.   |   172.73


Task 2.1: Fetch Additional Data Using Modules 10pts
- Allow the user to choose a module from the Quote Summary Endpoint.
Find:
  - 52 Week High
  - 52 Week Low
  - Return on Assets (ROA)
- Make an API request using the chosen module.
- Convert the response into a Pandas DataFrame.
- Display the DataFrame.

In [33]:
import pandas as pd

# I got confused here because it seems as though 52 week high, 52 week low, and
# ROA are in two different modules, not a random user-chosen module, meaning I can't
# find all 3 within the user-chosen module. I wasn't entirely sure what to do, so
# to get around it I just queried the API for the user's specific module along with
# the modules that contain the 3 things we were meant to find. I'm assuming this
# will be fine because it is still just using an API endpoint and sifting through
# the results.

print("Enter a stock symbol")
symbol = input()

print("Enter a module for " + symbol)
module = input()
all_modules = module + ",summaryDetail,financialData"

url = "https://yfapi.net/v11/finance/quoteSummary/" + symbol
querystring = {"modules": all_modules}

response = requests.request("GET", url, headers=headers, params=querystring)
response = response.json()["quoteSummary"]["result"][0]
user_module_df = pd.DataFrame(response[module])
print("\nHere is " + symbol + "'s " + module + " module")
print(user_module_df)

roa_fmt = response["financialData"]["returnOnAssets"]["fmt"]
fiftytwo_week_low = response["summaryDetail"]["fiftyTwoWeekLow"]["fmt"]
fiftytwo_week_high = response["summaryDetail"]["fiftyTwoWeekHigh"]["fmt"]

print("\nAdditionally, here is some other relavent data to " + symbol + "...")
print("\t-The 52 Week High for " + symbol + " was " + fiftytwo_week_high)
print("\t-The 52 Week Low for " + symbol + " was " + fiftytwo_week_low)
print("\t-The return on assests for " + symbol + " was " + roa_fmt)

Enter a stock symbol
AAPL
Enter a module for AAPL
price

Here is AAPL's price module
         maxAge preMarketChangePercent preMarketChange  preMarketTime  \
raw           1              -0.011334        -2.79999     1740580199   
fmt           1                 -1.13%           -2.80     1740580199   
longFmt       1                    NaN             NaN     1740580199   

        preMarketPrice preMarketSource postMarketChangePercent  \
raw             244.24   FREE_REALTIME               -0.003162   
fmt             244.24   FREE_REALTIME                  -0.32%   
longFmt            NaN   FREE_REALTIME                     NaN   

        postMarketChange  postMarketTime postMarketPrice  ... currency  \
raw            -0.759995      1740617991           239.6  ...      USD   
fmt                -0.76      1740617991          239.60  ...      USD   
longFmt              NaN      1740617991             NaN  ...      USD   

                quoteSourceName currencySymbol  fromCurrency

Task 2.2: Fetch Additional Data Using Modules 10pts
- Find Current Trending Stocks , Their Actual Name and Ticker and Current Price as well as
their 52 high and low

In [32]:
# have to specify region, going with the US for simplicity sake

url = "https://yfapi.net/v1/finance/trending/US"

response = requests.request("GET", url, headers=headers)
tickers = response.json()["finance"]["result"][0]["quotes"]

try:
    print("---------------------------------------------------------------------")
    print("TICKER  |        NAME           |  CURRENT MP |  52w High  |  52w Low")
    print("---------------------------------------------------------------------")
    for ticker in tickers:
        # get name + ticker + current price
        symbol = ticker["symbol"]
        quote_url = "https://yfapi.net/v6/finance/quote"
        querystring = {"symbols" : symbol}
        response = requests.request("GET", quote_url, headers=headers, params=querystring)

        response = response.json()["quoteResponse"]["result"][0]
        symbol, name, market_price = response["symbol"], response["longName"], response["regularMarketPrice"]

        # get 52 high and low
        quote_summary_url = "https://yfapi.net/v11/finance/quoteSummary/" + symbol
        querystring = {"modules": "summaryDetail"}
        response = requests.request("GET", quote_summary_url, headers=headers, params=querystring)
        response = response.json()["quoteSummary"]["result"][0]
        fiftytwo_week_low = response["summaryDetail"]["fiftyTwoWeekLow"]["fmt"]
        fiftytwo_week_high = response["summaryDetail"]["fiftyTwoWeekHigh"]["fmt"]

        print(symbol, "  |  ", name, "  |  ", market_price, "  |  ", fiftytwo_week_high, "  |  ", fiftytwo_week_low)
except Exception as error:
    print("A problem occured while fetching trending stocks. This is likely related to the limited queries my Yahoo Finance account gets.")
    print("Error: " + error)

---------------------------------------------------------------------
TICKER  |        NAME           |  CURRENT MP |  52w High  |  52w Low
---------------------------------------------------------------------
NVDA   |   NVIDIA Corporation   |   131.28   |   153.13   |   75.61
CRM   |   Salesforce, Inc.   |   307.33   |   369.00   |   212.00
SNOW   |   Snowflake Inc.   |   166.19   |   194.40   |   107.13
IONQ   |   IonQ, Inc.   |   29.93   |   54.74   |   6.22
BYND   |   Beyond Meat, Inc.   |   3.56   |   11.7000   |   3.3000
NIO   |   NIO Inc.   |   4.72   |   7.7100   |   3.6100
MRNA   |   Moderna, Inc.   |   33.58   |   170.47   |   29.25
MARA   |   MARA Holdings, Inc.   |   12.45   |   30.71   |   12.05
TDOC   |   Teladoc Health, Inc.   |   10.99   |   15.95   |   6.76
APP   |   AppLovin Corporation   |   331.0   |   525.15   |   57.40
NTNX   |   Nutanix, Inc.   |   69.35   |   75.80   |   43.35
AI   |   C3.ai, Inc.   |   26.44   |   45.08   |   18.85
PBR   |   Petróleo Brasileiro